# Before you use this template

This template is just a recommended template for project Report. It only considers the general type of research in our paper pool. Feel free to edit it to better fit your project. You will iteratively update the same notebook submission for your draft and the final submission. Please check the project rubriks to get a sense of what is expected in the template.

---

# FAQ and Attentions
* Copy and move this template to your Google Drive. Name your notebook by your team ID (upper-left corner). Don't eidt this original file.
* This template covers most questions we want to ask about your reproduction experiment. You don't need to exactly follow the template, however, you should address the questions. Please feel free to customize your report accordingly.
* any report must have run-able codes and necessary annotations (in text and code comments).
* The notebook is like a demo and only uses small-size data (a subset of original data or processed data), the entire runtime of the notebook including data reading, data process, model training, printing, figure plotting, etc,
must be within 8 min, otherwise, you may get penalty on the grade.
  * If the raw dataset is too large to be loaded  you can select a subset of data and pre-process the data, then, upload the subset or processed data to Google Drive and load them in this notebook.
  * If the whole training is too long to run, you can only set the number of training epoch to a small number, e.g., 3, just show that the training is runable.
  * For results model validation, you can train the model outside this notebook in advance, then, load pretrained model and use it for validation (display the figures, print the metrics).
* The post-process is important! For post-process of the results,please use plots/figures. The code to summarize results and plot figures may be tedious, however, it won't be waste of time since these figures can be used for presentation. While plotting in code, the figures should have titles or captions if necessary (e.g., title your figure with "Figure 1. xxxx")
* There is not page limit to your notebook report, you can also use separate notebooks for the report, just make sure your grader can access and run/test them.
* If you use outside resources, please refer them (in any formats). Include the links to the resources if necessary.

# Mount Notebook to Google Drive
Upload the data, pretrianed model, figures, etc to your Google Drive, then mount this notebook to Google Drive. After that, you can access the resources freely.

Instruction: https://colab.research.google.com/notebooks/io.ipynb

Example: https://colab.research.google.com/drive/1srw_HFWQ2SMgmWIawucXfusGzrj1_U0q

Video: https://www.youtube.com/watch?v=zc8g8lGcwQU

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# Introduction
The gut microbiome is a vital component of human health.$^1$ Recent evidence suggests it can be used to predict various diseases.$^2$ 16S rRNA gene sequencing technology can be used to profile the most common components of the human microbiome in a cost-effective way.$^3$ Similarly, a deeper, strain level, resolution profile of the microbial community can be obtained by shotgun metagenomic sequencing technology.$^4$ As the cost of obtaining microbiome data decreases, there is a novel opportunity for machine learning techniques to be employed for the early prediction of diseases. Detecting diseases early will not only decrease healthcare costs but also improve patient outcomes.

Microbiome datasets often contain samples in the range of 10-1000, while the data itself can have hundreds of thousands of dimensions. This poses a challenge to train machine learning models directly on the highly sparse data. The large number of features are computationally expensive and the relatively low number of samples makes the model less generalizabile to other datasets. As of April 2024, state-of-the-art techniques in this field make use of Conditional Generative Adversarial Networks (C-GANs)$^5$ to artificially augment the size of the dataset and Variational Information Bottlenecks (VIBs)$^{6,7}$ to extract only the relevant features for disease prediction while filtering out redundant information.

At the time of publication of the DeepMicro study$^8$, there had been little work on deep learning applications for microbiome data with a rigorous evaluation scheme. DeepMicro transforms high-dimensional microbiome data into a robust low-dimensional representation using an autoencoder and then applies machine learning classification on the learned representation. A thorough validation scheme optimizes hyper-parameters using a grid search, where the test set is excluded during cross-validation to ensure fairness. DeepMicro outperforms the current best approaches based on the strain-level marker profile$^9$ in five datasets, including IBD (AUC=0.955), EW-T2D (AUC=0.899), C-T2D (AUC=0.763), Obesity (AUC=0.659) and Cirrhosis (AUC=0.940). For the Colorectal dataset, DeepMicro has slightly lower performance than the best approach (DeepMicro's AUC=0.803 vs. MetAML's AUC=0.811) Additionally, reducing the dimensionality has sped up model training and hyperparameter tuning buy 8-30 times.$^8$

# Scope of Reproducibility:
- Hypothesis 1: Training classifiers using a lower dimensional representation will result in more accurate predictions.
- Hypothesis 2: Training classifiers using a lower dimensional representation will speed up the model training and hyperparameter tuning process.
- Hypothesis 3: Models trained on strain-level profiles will outperform those trained on abundance profiles.

# Methodology

This methodology is the core of your project. It consists of run-able codes with necessary annotations to show the expeiment you executed for testing the hypotheses.

The methodology at least contains two subsections **data** and **model** in your experiment.

In [2]:
!pip install python-docx
!pip install docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 439.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53895 sha256=d2d77e4f260d895cdcd43a8737e5c373787ad5ac786296bdc9049f010ed4596b
  Stored in directory: /root/.cache/pip/wheels/81/f5/1d/e09ba2c1907a43a4146d1189ae4733ca1a3bfe27ee39507767
Successfully built docx


In [3]:
# import packages you need
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from google.colab import drive

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# importing sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.random_projection import GaussianRandomProjection
from sklearn import cluster
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


# importing util libraries
import datetime
import time
import math
import os
import importlib

import sys

import exception_handle


##  Data
Data includes raw data (MIMIC III tables), descriptive statistics (our homework questions), and data processing (feature engineering).
  * Source of the data: where the data is collected from; if data is synthetic or self-generated, explain how. If possible, please provide a link to the raw datasets.
  * Statistics: include basic descriptive statistics of the dataset like size, cross validation split, label distribution, etc.
  * Data process: how do you munipulate the data, e.g., change the class labels, split the dataset to train/valid/test, refining the dataset.
  * Illustration: printing results, plotting figures for illustration.
  * You can upload your raw dataset to Google Drive and mount this Colab to the same directory. If your raw dataset is too large, you can upload the processed dataset and have a code to load the processed dataset.

In [4]:
# dir and function to load raw data
# raw_data_dir = '/content/gdrive/My Drive/Colab Notebooks/<path-to-raw-data>'

raw_data_dir = '/content/data'


def load_raw_data(raw_data_dir):
  # implement this function to load raw data to dataframe/numpy array/tensor
  return None

raw_data = load_raw_data(raw_data_dir)

# calculate statistics
def calculate_stats(raw_data):
  # implement this function to calculate the statistics
  # it is encouraged to print out the results
  return None

# process raw data
def process_data(raw_data):
    # implement this function to process the data as you need
  return None

processed_data = process_data(raw_data)

''' you can load the processed data directly
processed_data_dir = '/content/gdrive/My Drive/Colab Notebooks/<path-to-raw-data>'
def load_processed_data(raw_data_dir):
  pass

'''

" you can load the processed data directly\nprocessed_data_dir = '/content/gdrive/My Drive/Colab Notebooks/<path-to-raw-data>'\ndef load_processed_data(raw_data_dir):\n  pass\n\n"

##   Model
The model includes the model definitation which usually is a class, model training, and other necessary parts.
  * Model architecture: layer number/size/type, activation function, etc
  * Training objectives: loss function, optimizer, weight of each loss term, etc
  * Others: whether the model is pretrained, Monte Carlo simulation for uncertainty analysis, etc
  * The code of model should have classes of the model, functions of model training, model validation, etc.
  * If your model training is done outside of this notebook, please upload the trained model here and develop a function to load and test it.

## Autoencoders

To reproduce the paper we needed an Autoencoder, Convolutional AutoEnconder and Variational Autoenconder

In [5]:
class Autoencoder(nn.Module):
    def __init__(self, dims, act=nn.ReLU(), latent_act=False, output_act=nn.Sigmoid()):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential()
        for i in range(len(dims) - 1):
            self.encoder.add_module(f"encoder_{i}", nn.Linear(dims[i], dims[i+1]))
            if i < len(dims) - 2 or latent_act:
                self.encoder.add_module(f"act_{i}", act)

        self.decoder = nn.Sequential()
        for i in range(len(dims) - 1, 0, -1):
            self.decoder.add_module(f"decoder_{len(dims)-i-1}", nn.Linear(dims[i], dims[i-1]))
            if i > 1:
                self.decoder.add_module(f"act_{len(dims)-i-1}", act)

        if output_act is not None:
            self.decoder.add_module("output_act", output_act)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded



class ConvAutoencoder(nn.Module):
    def __init__(self, channels, kernel_sizes, strides, paddings, latent_dims):
        super(ConvAutoencoder, self).__init__()
        self.encoder = nn.Sequential()
        for i in range(len(channels) - 1):
            self.encoder.add_module(f"conv_{i}", nn.Conv2d(channels[i], channels[i+1], kernel_sizes[i], strides[i], paddings[i]))
            self.encoder.add_module(f"relu_{i}", nn.ReLU(True))

        self.decoder = nn.Sequential()
        rev_channels = channels[::-1]
        rev_kernel_sizes = kernel_sizes[::-1]
        rev_strides = strides[::-1]
        rev_paddings = paddings[::-1]
        for i in range(len(rev_channels) - 1):
            self.decoder.add_module(f"deconv_{i}", nn.ConvTranspose2d(rev_channels[i], rev_channels[i+1], rev_kernel_sizes[i], rev_strides[i], rev_paddings[i]))
            self.decoder.add_module(f"relu_{i}", nn.ReLU(True))

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x



class VariationalAutoencoder(nn.Module):
    def __init__(self, input_dims, hidden_dims, latent_dims, device='cpu'):
        super(VariationalAutoencoder, self).__init__()
        self.device = torch.device(device)
        self.input_dims = input_dims
        self.fc1 = nn.Linear(input_dims, hidden_dims).to(self.device)
        self.fc2_mean = nn.Linear(hidden_dims, latent_dims).to(self.device)
        self.fc2_logvar = nn.Linear(hidden_dims, latent_dims).to(self.device)
        self.fc3 = nn.Linear(latent_dims, hidden_dims).to(self.device)
        self.fc4 = nn.Linear(hidden_dims, input_dims).to(self.device)

    def encode(self, x):
        x = torch.tensor(x, dtype=torch.float32)
        h1 = F.relu(self.fc1(x))
        return self.fc2_mean(h1), self.fc2_logvar(h1)

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        # Adjust x if necessary (handled outside the model)
        mean, logvar = self.encode(x.to(self.device))
        z = self.reparameterize(mean, logvar)
        return self.decode(z), mean, logvar


def vae_loss(recon_x, x, mu, logvar):
    # Assuming x is your input tensor and it originally has a shape compatible with [batch_size, 200]
    # You need to ensure recon_x and x have the same shape for BCE calculation
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')

    # Calculation of KL Divergence remains the same
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

## DeepMicrobiome Framework

This code was translated from Tensorflow to Pytorch



In [6]:
class DeepMicrobiome(object):
    def __init__(self, data, seed, data_dir):
        self.t_start = time.time()
        self.filename = str(data)
        self.data = self.filename.split('.')[0]
        self.seed = seed
        self.data_dir = data_dir
        self.prefix = ''
        self.representation_only = False

    def loadData(self, feature_string, label_string, label_dict, dtype=None):
        # read file
        filename = self.data_dir + "data/" + self.filename
        if os.path.isfile(filename):
            raw = pd.read_csv(filename, sep='\t', index_col=0, header=None)
        else:
            print("FileNotFoundError: File {} does not exist".format(filename))
            exit()

        # select rows having feature index identifier string
        X = raw.loc[raw.index.str.contains(feature_string, regex=False)].T

        # get class labels
        Y = raw.loc[label_string] #'disease'
        Y = Y.replace(label_dict)

        # train and test split
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X.values.astype(dtype), Y.values.astype('int'), test_size=0.2, random_state=self.seed, stratify=Y.values)
        if isinstance(self.X_train, np.ndarray):
            self.X_train = torch.tensor(self.X_train, dtype=torch.float32)
            self.X_test= torch.tensor(self.X_test, dtype=torch.float32)
            self.y_train = torch.tensor(self.y_train, dtype=torch.float32)
            self.y_test = torch.tensor(self.y_test, dtype=torch.float32)

        self.printDataShapes()

    def loadCustomData(self, dtype=None):
        # read file
        filename = self.data_dir + "data/" + self.filename
        if os.path.isfile(filename):
            raw = pd.read_csv(filename, sep=',', index_col=False, header=None)
        else:
            print("FileNotFoundError: File {} does not exist".format(filename))
            exit()

        # load data
        self.X_train = raw.values.astype(dtype)

        # put nothing or zeros for y_train, y_test, and X_test
        self.y_train = np.zeros(shape=(self.X_train.shape[0])).astype(dtype)
        self.X_test = np.zeros(shape=(1,self.X_train.shape[1])).astype(dtype)
        self.y_test = np.zeros(shape=(1,)).astype(dtype)
        self.printDataShapes(train_only=True)

    def loadCustomDataWithLabels(self, label_data, dtype=None):
        # read file
        filename = self.data_dir + "data/" + self.filename
        label_filename = self.data_dir + "data/" + label_data
        if os.path.isfile(filename) and os.path.isfile(label_filename):
            raw = pd.read_csv(filename, sep=',', index_col=False, header=None)
            label = pd.read_csv(label_filename, sep=',', index_col=False, header=None)
        else:
            if not os.path.isfile(filename):
                print("FileNotFoundError: File {} does not exist".format(filename))
            if not os.path.isfile(label_filename):
                print("FileNotFoundError: File {} does not exist".format(label_filename))
            exit()

        # label data validity check
        if not label.values.shape[1] > 1:
            label_flatten = label.values.reshape((label.values.shape[0]))
        else:
            print('FileSpecificationError: The label file contains more than 1 column.')
            exit()

        # train and test split
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(raw.values.astype(dtype),
                                                                                label_flatten.astype('int'), test_size=0.2,
                                                                                random_state=self.seed,
                                                                                stratify=label_flatten)
        self.printDataShapes()


    #Principal Component Analysis
    def pca(self, ratio=0.99):
        # manipulating an experiment identifier in the output file
        self.prefix = self.prefix + 'PCA_'

        # PCA
        pca = PCA()
        pca.fit(self.X_train)
        n_comp = 0
        ratio_sum = 0.0

        for comp in pca.explained_variance_ratio_:
            ratio_sum += comp
            n_comp += 1
            if ratio_sum >= ratio:  # Selecting components explaining 99% of variance
                break

        pca = PCA(n_components=n_comp)
        pca.fit(self.X_train)

        X_train = pca.transform(self.X_train)
        X_test = pca.transform(self.X_test)

        # applying the eigenvectors to the whole training and the test set.
        self.X_train = X_train
        self.X_test = X_test
        self.printDataShapes()

    #Gausian Random Projection
    def rp(self):
        # manipulating an experiment identifier in the output file
        self.prefix = self.prefix + 'RandP_'
        # GRP
        rf = GaussianRandomProjection(eps=0.5)
        rf.fit(self.X_train)

        # applying GRP to the whole training and the test set.
        self.X_train = rf.transform(self.X_train)
        self.X_test = rf.transform(self.X_test)
        self.printDataShapes()

    #Shallow Autoencoder & Deep Autoencoder
    def ae(self, dims=[50], epochs=2000, batch_size=100, verbose=2, loss='mse', latent_act=False, output_act=False, act='relu', patience=20, val_rate=0.2, no_trn=False):
        self.prefix += 'AE_' if len(dims) == 1 else 'DAE_'
        self.prefix += f'{dims}_'

        # Adjustments for the PyTorch ecosystem
        if loss == 'mse':
            criterion = nn.MSELoss()
        elif loss == 'binary_crossentropy':
            criterion = nn.BCELoss()
            self.prefix += 'b'
        if act == 'sigmoid':
            self.prefix += 's'

        if isinstance(self.X_train, np.ndarray):
            self.X_train = torch.tensor(self.X_train, dtype=torch.float32)
            self.X_test = torch.tensor(self.X_test, dtype=torch.float32)

        # Setting up the model
        input_dim = self.X_train.shape[1]
        dims.insert(0, input_dim)  # Ensure the input layer dimension is included
        model = Autoencoder(dims=dims, act=nn.ReLU() if act == 'relu' else nn.Sigmoid(), latent_act=latent_act, output_act=nn.Sigmoid() if output_act else None)

        optimizer = optim.Adam(model.parameters())

        # Prepare data loaders
        X_inner_train, X_inner_test, _, _ = train_test_split(self.X_train, self.y_train, test_size=val_rate, random_state=self.seed, stratify=self.y_train)
        train_loader = DataLoader(TensorDataset(X_inner_train, X_inner_train), batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(TensorDataset(X_inner_test, X_inner_test), batch_size=batch_size, shuffle=False)

        # Early stopping criteria
        best_loss = np.inf
        patience_counter = 0

        for epoch in range(epochs):
            model.train()
            running_loss = 0.0
            for data in train_loader:
                inputs, labels = data
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * inputs.size(0)
            epoch_loss = running_loss / len(train_loader.dataset)

            # Validation
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for data in val_loader:
                    inputs, labels = data
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item() * inputs.size(0)
            val_loss /= len(val_loader.dataset)

            if verbose:
                print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}')

            # Check early stopping criteria
            if val_loss < best_loss:
                best_loss = val_loss
                patience_counter = 0
                # Save model if this is your best model so far
                torch.save(model.state_dict(), self.prefix + self.data + '.pt')
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print('Early stopping!')
                    break

        # Load the best model
        model.load_state_dict(torch.load(self.prefix + self.data + '.pt'))

        # Transform the training and test set
        self.X_train = model.encoder(self.X_train).detach()
        self.X_test = model.encoder(self.X_test).detach()

        if no_trn:
            return


    def vae(self, dims=[10], epochs=2000, batch_size=100, verbose=2, loss='mse', output_act=False, act='relu', patience=25, beta=1.0, warmup=True, warmup_rate=0.01, val_rate=0.2, no_trn=False):
        self.prefix = 'VAE_' + (''.join(str(dims)) + '_').replace(',', '-').replace(' ', '').replace('[', '').replace(']', '')
        if loss == 'binary_crossentropy':
            self.prefix += 'b_'
        if output_act:
            self.prefix += 'T_'
        if beta != 1:
            self.prefix += f'B{beta}_'
        if act == 'sigmoid':
            self.prefix += 'sig_'

        # Adjust for PyTorch
        input_dim = self.X_train.shape[1]
        model = VariationalAutoencoder(input_dim, dims[0], dims[-1])
        optimizer = optim.Adam(model.parameters())

        # Prepare data loaders
        X_inner_train, X_inner_test, _, _ = train_test_split(self.X_train.cpu().numpy(), self.y_train.cpu().numpy(), test_size=val_rate, random_state=self.seed, stratify=self.y_train.cpu().numpy())
        train_dataset = TensorDataset(torch.tensor(X_inner_train, dtype=torch.float), torch.tensor(X_inner_train, dtype=torch.float))
        val_dataset = TensorDataset(torch.tensor(X_inner_test, dtype=torch.float), torch.tensor(X_inner_test, dtype=torch.float))
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        best_loss = np.inf
        patience_counter = 0

        beta_val = 0.0 if warmup else beta  # Initialize beta for warm-up

        for epoch in range(epochs):
            model.train()
            train_loss = 0
            for batch_idx, (data, _) in enumerate(train_loader):
                data = data.to(model.fc1.weight.device)  # Ensure data is on the correct device
                optimizer.zero_grad()
                recon_batch, mu, logvar = model(data)
                loss = vae_loss(recon_batch, data, mu, logvar, beta=beta_val)
                loss.backward()
                train_loss += loss.item()
                optimizer.step()

                if warmup:  # Update beta_val if in warmup phase
                    beta_val = min(beta, beta_val + warmup_rate)

            # Validation
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for data, _ in val_loader:
                    data = data.to(model.fc1.weight.device)
                    recon_batch, mu, logvar = model(data)
                    loss = vae_loss(recon_batch, data, mu, logvar, beta=beta_val)
                    val_loss += loss.item()

            if verbose:
                print(f'Epoch {epoch + 1}, Train Loss: {train_loss / len(train_loader):.4f}, Val Loss: {val_loss / len(val_loader):.4f}')

            # Early Stopping check
            if val_loss < best_loss:
                best_loss = val_loss
                patience_counter = 0
                # Save best model
                torch.save(model.state_dict(), os.path.join(self.data_dir, modelName))
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    if verbose:
                        print("Early stopping")
                    break

        if not no_trn:
            # Load the best model
            model.load_state_dict(torch.load(os.path.join(self.data_dir, modelName)))
            model.eval()

            # Update training and test set using the encoder part of VAE
            with torch.no_grad():
                self.X_train = model.encode(self.X_train)[0]  # Only keep the mu component
                self.X_test = model.encode(self.X_test)[0]





    # Variational Autoencoder
    def vae(self, dims=[10], epochs=2000, batch_size=100, verbose=2, loss='mse', output_act=False, act='relu', patience=25, beta=1.0, warmup=True, warmup_rate=0.01, val_rate=0.2, no_trn=False):
        self.prefix = 'VAE_' + (''.join(str(dims)) + '_').replace(',', '-').replace(' ', '').replace('[', '').replace(']', '')
        if loss == 'binary_crossentropy':
            self.prefix += 'b_'
        if output_act:
            self.prefix += 'T_'
        if beta != 1:
            self.prefix += f'B{beta}_'
        if act == 'sigmoid':
            self.prefix += 'sig_'

        # Adjust for PyTorch
        input_dim = self.X_train.shape[1]
        model = VariationalAutoencoder(input_dim, dims[0], dims[-1])
        optimizer = optim.Adam(model.parameters())

        # Prepare data loaders
        X_inner_train, X_inner_test, _, _ = train_test_split(self.X_train, self.y_train, test_size=val_rate, random_state=self.seed, stratify=self.y_train)

        train_dataset = TensorDataset(torch.tensor(X_inner_train, dtype=torch.float), torch.tensor(X_inner_train, dtype=torch.float))
        val_dataset = TensorDataset(torch.tensor(X_inner_test, dtype=torch.float), torch.tensor(X_inner_test, dtype=torch.float))
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        best_loss = np.inf
        patience_counter = 0

        beta_val = 0.0 if warmup else beta  # Initialize beta for warm-up

        for epoch in range(epochs):
            model.train()
            train_loss = 0
            for batch_idx, (data, _) in enumerate(train_loader):
                data = data.to(model.fc1.weight.device)  # Ensure data is on the correct device
                optimizer.zero_grad()
                recon_batch, mu, logvar = model(data)
                loss = vae_loss(recon_batch, data, mu, logvar)
                loss.backward()
                train_loss += loss.item()
                optimizer.step()

                if warmup:  # Update beta_val if in warmup phase
                    beta_val = min(beta, beta_val + warmup_rate)

            # Validation
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for data, _ in val_loader:
                    data = data.to(model.fc1.weight.device)
                    recon_batch, mu, logvar = model(data)
                    loss = vae_loss(recon_batch, data, mu, logvar)
                    val_loss += loss.item()

            if verbose:
                print(f'Epoch {epoch + 1}, Train Loss: {train_loss / len(train_loader):.4f}, Val Loss: {val_loss / len(val_loader):.4f}')

            # Early Stopping check
            if val_loss < best_loss:
                best_loss = val_loss
                patience_counter = 0
                # Save best model
                torch.save(model.state_dict(), os.path.join(self.data_dir, 'vae'))
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    if verbose:
                        print("Early stopping")
                    break

        if not no_trn:
            # Load the best model
            model.load_state_dict(torch.load(os.path.join(self.data_dir, 'vae')))
            model.eval()

            # Update training and test set using the encoder part of VAE
            with torch.no_grad():
                self.X_train = model.encode(self.X_train)[0]  # Only keep the mu component
                self.X_test = model.encode(self.X_test)[0]


    # Convolutional Autoencoder
    def cae(self, dims=[32], epochs=2000, batch_size=100, verbose=2, loss='mse', output_act=False, act='relu', patience=25, val_rate=0.2, rf_rate=0.1, st_rate=0.25, no_trn=False):
        self.prefix += 'CAE_'
        if loss == 'binary_crossentropy':
            self.prefix += 'b_'
        if output_act:
            self.prefix += 'T_'
        self.prefix += f'{dims}_'
        if act == 'sigmoid':
            self.prefix += 'sig_'

        if isinstance(self.X_train, np.ndarray):
            self.X_train = torch.tensor(self.X_train, dtype=torch.float32)
            self.X_test = torch.tensor(self.X_test, dtype=torch.float32)

        # Normalize the data if not already normalized
        self.X_train /= 255.0
        self.X_test /= 255.0

        one_side_dim = int(math.sqrt(self.X_train.shape[1])) + 1
        enlarged_dim = one_side_dim ** 2
        padding = (0, enlarged_dim - self.X_train.shape[1])

        X_train_padded = torch.nn.functional.pad(self.X_train, padding).view(-1, 1, one_side_dim, one_side_dim)
        X_test_padded = torch.nn.functional.pad(self.X_test, padding).view(-1, 1, one_side_dim, one_side_dim)

        train_dataset = TensorDataset(X_train_padded, X_train_padded)
        val_dataset = TensorDataset(X_test_padded, X_test_padded)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        channels = [1] + dims
        kernel_sizes = [max(1, int(one_side_dim * rf_rate))] * len(dims)
        strides = [max(1, int(kernel_sizes[0] * st_rate))] * len(dims)
        paddings = [0] * len(dims)

        model = ConvAutoencoder(channels=channels, kernel_sizes=kernel_sizes, strides=strides, paddings=paddings, latent_dims=dims[-1])
        criterion = nn.MSELoss() if loss == 'mse' else nn.BCELoss()
        optimizer = optim.Adam(model.parameters())

        best_loss = float('inf')
        for epoch in range(epochs):
            model.train()
            running_loss = 0.0
            for data, target in train_loader:
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * data.size(0)
                # Print gradient norms for debugging
                for name, param in model.named_parameters():
                    if param.grad is not None:
                        print(f"Gradient norm for {name}: {torch.norm(param.grad)}")

            running_loss /= len(train_loader.dataset)

            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for data, target in val_loader:
                    output = model(data)
                    val_loss += criterion(output, target).item() * data.size(0)
            val_loss /= len(val_loader.dataset)

            if verbose > 1:
                print(f'Epoch {epoch+1}, Train Loss: {running_loss:.4f}, Val Loss: {val_loss:.4f}')

            if val_loss < best_loss:
                best_loss = val_loss
                best_model_wts = model.state_dict()

    # Classification
    def classification(self, hyper_parameters, method='svm', cv=5, scoring='roc_auc', n_jobs=1, cache_size=10000):
        clf_start_time = time.time()

        print("# Tuning hyper-parameters")
        print(self.X_train.shape, self.y_train.shape)

        # Support Vector Machine
        if method == 'svm':
            clf = GridSearchCV(SVC(probability=True, cache_size=cache_size), hyper_parameters, cv=StratifiedKFold(cv, shuffle=True), scoring=scoring, n_jobs=n_jobs, verbose=100, )
            clf.fit(self.X_train, self.y_train)

        # Random Forest
        if method == 'rf':
            clf = GridSearchCV(RandomForestClassifier(n_jobs=-1, random_state=0), hyper_parameters, cv=StratifiedKFold(cv, shuffle=True), scoring=scoring, n_jobs=n_jobs, verbose=100)
            clf.fit(self.X_train, self.y_train)

        # Multi-layer Perceptron
        if method == 'mlp':
            model = KerasClassifier(build_fn=mlp_model, input_dim=self.X_train.shape[1], verbose=0, )
            clf = GridSearchCV(estimator=model, param_grid=hyper_parameters, cv=StratifiedKFold(cv, shuffle=True), scoring=scoring, n_jobs=n_jobs, verbose=100)
            clf.fit(self.X_train, self.y_train, batch_size=32)

        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)

        # Evaluate performance of the best model on test set
        y_true, y_pred = self.y_test, clf.predict(self.X_test)
        y_prob = clf.predict_proba(self.X_test)

        # Performance Metrics: AUC, ACC, Recall, Precision, F1_score
        metrics = [round(roc_auc_score(y_true, y_prob[:, 1]), 4),
                   round(accuracy_score(y_true, y_pred), 4),
                   round(recall_score(y_true, y_pred), 4),
                   round(precision_score(y_true, y_pred), 4),
                   round(f1_score(y_true, y_pred), 4), ]

        # time stamp
        metrics.append(str(datetime.datetime.now()))

        # running time
        metrics.append(round( (time.time() - self.t_start), 2))

        # classification time
        metrics.append(round( (time.time() - clf_start_time), 2))

        # best hyper-parameter append
        metrics.append(str(clf.best_params_))

        # Write performance metrics as a file
        res = pd.DataFrame([metrics], index=[self.prefix + method])
        with open(self.data_dir + "results/" + self.data + "_result.txt", 'a') as f:
            res.to_csv(f, header=None)

        print('Accuracy metrics')
        print('AUC, ACC, Recall, Precision, F1_score, time-end, runtime(sec), classfication time(sec), best hyper-parameter')
        print(metrics)

    def printDataShapes(self, train_only=False):
        print("X_train.shape: ", self.X_train.shape)
        if not train_only:
            print("y_train.shape: ", self.y_train.shape)
            print("X_test.shape: ", self.X_test.shape)
            print("y_test.shape: ", self.y_test.shape)

    # ploting loss progress over epochs
    def saveLossProgress(self):
        #print(self.history.history.keys())
        #print(type(self.history.history['loss']))
        #print(min(self.history.history['loss']))

        loss_collector, loss_max_atTheEnd = self.saveLossProgress_ylim()

        # save loss progress - train and val loss only
        figureName = self.prefix + self.data + '_' + str(self.seed)
        plt.ylim(min(loss_collector)*0.9, loss_max_atTheEnd * 2.0)
        plt.plot(self.history.history['loss'])
        plt.plot(self.history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train loss', 'val loss'],
                   loc='upper right')
        plt.savefig(self.data_dir + "results/" + figureName + '.png')
        plt.close()

        if 'recon_loss' in self.history.history:
            figureName = self.prefix + self.data + '_' + str(self.seed) + '_detailed'
            plt.ylim(min(loss_collector) * 0.9, loss_max_atTheEnd * 2.0)
            plt.plot(self.history.history['loss'])
            plt.plot(self.history.history['val_loss'])
            plt.plot(self.history.history['recon_loss'])
            plt.plot(self.history.history['val_recon_loss'])
            plt.plot(self.history.history['kl_loss'])
            plt.plot(self.history.history['val_kl_loss'])
            plt.title('model loss')
            plt.ylabel('loss')
            plt.xlabel('epoch')
            plt.legend(['train loss', 'val loss', 'recon_loss', 'val recon_loss', 'kl_loss', 'val kl_loss'], loc='upper right')
            plt.savefig(self.data_dir + "results/" + figureName + '.png')
            plt.close()

    # supporting loss plot
    def saveLossProgress_ylim(self):
        loss_collector = []
        loss_max_atTheEnd = 0.0
        for hist in self.history.history:
            current = self.history.history[hist]
            loss_collector += current
            if current[-1] >= loss_max_atTheEnd:
                loss_max_atTheEnd = current[-1]
        return loss_collector, loss_max_atTheEnd

In [7]:
import sys

# Save the original argv
original_argv = sys.argv

# Set the parameters to run the experiment

# Experiment 1:
#sys.argv = ['DM.py', '-r', '1', '--no_clf', '-cd', 'UserDataExample.csv', '--ae', '-dm', '20', '--save_rep']

# Experiment 2:
# 4. Suppose that we want to use deep autoencoder with 2 hidden layers which has 100 units and 40 units,
# respectively. Let the size of latent layer to be 20. We are going to see the structure of deep autoencoder first.

#sys.argv = ['DM.py', '-r', '1', '--no_clf', '-cd', 'UserDataExample.csv', '--ae', '-dm', '100,40,20', '--no_trn']

# Now, run the model and get the learned representation.

#sys.argv = ['DM.py', '-r', '1', '--no_clf', '-cd', 'UserDataExample.csv', '--ae', '-dm', '100,40,20', '--save_rep']

# Experiment 3:
# variational autoencoder
sys.argv = ['DM.py', '-r', '1', '--no_clf', '-cd', 'UserDataExample.csv', '--vae', '-dm', '100,20', '--save_rep']

# Experiment 4:
# Convolutional Autoencoder
#sys.argv = ['DM.py', '-r', '1', '--no_clf', '-cd', 'UserDataExample.csv', '--cae', '-dm', '100,50,1', '--save_rep']

#
sys.argv = ['DM.py', '-r', '1', '--no_clf', '-cd', 'UserDataExample.csv', '-cl', 'UserLabelExample.csv', '-m', 'svm']


# Now you can run your argument parsing code

# argparse
import argparse
parser = argparse.ArgumentParser()
parser._action_groups.pop()
# load data
load_data = parser.add_argument_group('Loading data')
load_data.add_argument("-d", "--data", help="prefix of dataset to open (e.g. abundance_Cirrhosis)", type=str,
                    choices=["abundance_Cirrhosis", "abundance_Colorectal", "abundance_IBD",
                             "abundance_Obesity", "abundance_T2D", "abundance_WT2D",
                             "marker_Cirrhosis", "marker_Colorectal", "marker_IBD",
                             "marker_Obesity", "marker_T2D", "marker_WT2D",
                             ])
load_data.add_argument("-cd", "--custom_data", help="filename for custom input data under the 'data' folder", type=str,)
load_data.add_argument("-cl", "--custom_data_labels", help="filename for custom input labels under the 'data' folder", type=str,)
load_data.add_argument("-p", "--data_dir", help="custom path for both '/data' and '/results' folders", default="")
load_data.add_argument("-dt", "--dataType", help="Specify data type for numerical values (float16, float32, float64)",
                    default="float64", type=str, choices=["float16", "float32", "float64"])
dtypeDict = {"float16": np.float16, "float32": np.float32, "float64": np.float64}
# experiment design
exp_design = parser.add_argument_group('Experiment design')
exp_design.add_argument("-s", "--seed", help="random seed for train and test split", type=int, default=0)
exp_design.add_argument("-r", "--repeat", help="repeat experiment x times by changing random seed for splitting data",
                    default=5, type=int)
# classification
classification = parser.add_argument_group('Classification')
classification.add_argument("-f", "--numFolds", help="The number of folds for cross-validation in the tranining set",
                    default=5, type=int)
classification.add_argument("-m", "--method", help="classifier(s) to use", type=str, default="all",
                    choices=["all", "svm", "rf", "mlp", "svm_rf"])
classification.add_argument("-sc", "--svm_cache", help="cache size for svm run", type=int, default=1000)
classification.add_argument("-t", "--numJobs",
                    help="The number of jobs used in parallel GridSearch. (-1: utilize all possible cores; -2: utilize all possible cores except one.)",
                    default=-2, type=int)
parser.add_argument("--scoring", help="Metrics used to optimize method", type=str, default='roc_auc',
                    choices=['roc_auc', 'accuracy', 'f1', 'recall', 'precision'])
# representation learning & dimensionality reduction algorithms
rl = parser.add_argument_group('Representation learning')
rl.add_argument("--pca", help="run PCA", action='store_true')
rl.add_argument("--rp", help="run Random Projection", action='store_true')
rl.add_argument("--ae", help="run Autoencoder or Deep Autoencoder", action='store_true')
rl.add_argument("--vae", help="run Variational Autoencoder", action='store_true')
rl.add_argument("--cae", help="run Convolutional Autoencoder", action='store_true')
rl.add_argument("--save_rep", help="write the learned representation of the training set as a file", action='store_true')
# detailed options for representation learning
## common options
common = parser.add_argument_group('Common options for representation learning (SAE,DAE,VAE,CAE)')
common.add_argument("--aeloss", help="set autoencoder reconstruction loss function", type=str,
                    choices=['mse', 'binary_crossentropy'], default='mse')
common.add_argument("--ae_oact", help="output layer sigmoid activation function on/off", action='store_true')
common.add_argument("-a", "--act", help="activation function for hidden layers", type=str, default='relu',
                    choices=['relu', 'sigmoid'])
common.add_argument("-dm", "--dims",
                    help="Comma-separated dimensions for deep representation learning e.g. (-dm 50,30,20)",
                    type=str, default='50')
common.add_argument("-e", "--max_epochs", help="Maximum epochs when training autoencoder", type=int, default=2000)
common.add_argument("-pt", "--patience",
                    help="The number of epochs which can be executed without the improvement in validation loss, right after the last improvement.",
                    type=int, default=20)
## AE & DAE only
AE = parser.add_argument_group('SAE & DAE-specific arguments')
AE.add_argument("--ae_lact", help="latent layer activation function on/off", action='store_true')
## VAE only
VAE = parser.add_argument_group('VAE-specific arguments')
VAE.add_argument("--vae_beta", help="weight of KL term", type=float, default=1.0)
VAE.add_argument("--vae_warmup", help="turn on warm up", action='store_true')
VAE.add_argument("--vae_warmup_rate", help="warm-up rate which will be multiplied by current epoch to calculate current beta", default=0.01, type=float)
## CAE only
CAE = parser.add_argument_group('CAE-specific arguments')
CAE.add_argument("--rf_rate", help="What percentage of input size will be the receptive field (kernel) size? [0,1]", type=float, default=0.1)
CAE.add_argument("--st_rate", help="What percentage of receptive field (kernel) size will be the stride size? [0,1]", type=float, default=0.25)
# other options
others = parser.add_argument_group('other optional arguments')
others.add_argument("--no_trn", help="stop before learning representation to see specified autoencoder structure", action='store_true')
others.add_argument("--no_clf", help="skip classification tasks", action='store_true')
args = parser.parse_args()
print(args)
# set labels for diseases and controls
label_dict = {
    # Controls
    'n': 0,
    # Chirrhosis
    'cirrhosis': 1,
    # Colorectal Cancer
      'cancer': 1, 'small_adenoma': 0,
      # IBD
      'ibd_ulcerative_colitis': 1, 'ibd_crohn_disease': 1,
      # T2D and WT2D
      't2d': 1,
      # Obesity
      'leaness': 0, 'obesity': 1,
  }
# hyper-parameter grids for classifiers
rf_hyper_parameters = [{'n_estimators': [s for s in range(100, 1001, 200)],
                        'max_features': ['sqrt', 'log2'],
                        'min_samples_leaf': [1, 2, 3, 4, 5],
                        'criterion': ['gini', 'entropy']
                        }, ]
#svm_hyper_parameters_pasolli = [{'C': [2 ** s for s in range(-5, 16, 2)], 'kernel': ['linear']},
#                        {'C': [2 ** s for s in range(-5, 16, 2)], 'gamma': [2 ** s for s in range(3, -15, -2)],
#                         'kernel': ['rbf']}]
svm_hyper_parameters = [{'C': [2 ** s for s in range(-5, 6, 2)], 'kernel': ['linear']},
                        {'C': [2 ** s for s in range(-5, 6, 2)], 'gamma': [2 ** s for s in range(3, -15, -2)],'kernel': ['rbf']}]
mlp_hyper_parameters = [{'numHiddenLayers': [1, 2, 3],
                         'epochs': [30, 50, 100, 200, 300],
                         'numUnits': [10, 30, 50, 100],
                         'dropout_rate': [0.1, 0.3],
                         },]


Namespace(data=None, custom_data='UserDataExample.csv', custom_data_labels='UserLabelExample.csv', data_dir='', dataType='float64', seed=0, repeat=1, numFolds=5, method='svm', svm_cache=1000, numJobs=-2, scoring='roc_auc', pca=False, rp=False, ae=False, vae=False, cae=False, save_rep=False, aeloss='mse', ae_oact=False, act='relu', dims='50', max_epochs=2000, patience=20, ae_lact=False, vae_beta=1.0, vae_warmup=False, vae_warmup_rate=0.01, rf_rate=0.1, st_rate=0.25, no_trn=False, no_clf=True)


# Classification
1. The function now supports running multiple classifiers based on the `method` argument. If `method` is set to 'all' or 'svm_rf', it will run the specified classifiers (SVM, Random Forest, and MLP for 'all', or SVM and Random Forest for 'svm_rf'). If a single classifier is specified, it will run only that classifier.

2. The function uses the `GridSearchCV` from scikit-learn to perform hyperparameter tuning for each classifier. It uses the specified hyperparameter grid (`hyper_parameters`) and cross-validation settings (`cv`, `scoring`, `n_jobs`).

3. For each classifier, the function fits the model on the training data, predicts the labels and probabilities for the test data, and calculates various evaluation metrics (AUC, accuracy, recall, precision, F1-score).

In [8]:
# run exp function
def run_exp(seed):
    # Initialize the DeepMicrobiome instance
    data_file = args.data + '.txt' if args.data else args.custom_data
    dm = DeepMicrobiome(data=data_file, seed=seed, data_dir=args.data_dir)
            # Load data based on the specified dataset type
    if args.data:
        feature_string = "k__" if "abundance" in args.data else "gi|" if "marker" in args.data else ''
        dm.loadData(feature_string=feature_string, label_string='disease', label_dict=label_dict, dtype=dtypeDict[args.dataType])
    elif args.custom_data:
        if args.custom_data_labels:
            dm.loadCustomDataWithLabels(label_data=args.custom_data_labels, dtype=dtypeDict[args.dataType])
        else:
            dm.loadCustomData(dtype=dtypeDict[args.dataType])
    else:
        print("[Error] No input file specified. Use -h for help.")
        exit()
            # Representation learning (Dimensionality reduction)
    if args.pca:
        dm.pca()
    if args.ae:
        dm.ae(dims=[int(i) for i in args.dims.split(',')], act=args.act, epochs=args.max_epochs, loss=args.aeloss,
              latent_act=args.ae_lact, output_act=args.ae_oact, patience=args.patience, no_trn=args.no_trn)
    if args.vae:
        dm.vae(dims=[int(i) for i in args.dims.split(',')], act=args.act, epochs=args.max_epochs, loss=args.aeloss, output_act=args.ae_oact,
               patience=25 if args.patience == 20 else args.patience, beta=args.vae_beta, warmup=args.vae_warmup, warmup_rate=args.vae_warmup_rate, no_trn=args.no_trn)
    if args.cae:
        dm.cae(dims=[int(i) for i in args.dims.split(',')], act=args.act, epochs=args.max_epochs, loss=args.aeloss, output_act=args.ae_oact,
               patience=args.patience, rf_rate=args.rf_rate, st_rate=args.st_rate, no_trn=args.no_trn)
    if args.rp:
        dm.rp()
            # Write the learned representation to a file if required
    if args.save_rep and (args.pca or args.ae or args.rp or args.vae or args.cae):
        rep_file = f"{dm.data_dir}results/{dm.prefix}{dm.data}_rep.csv"
        X_train_flat = dm.X_train.view(dm.X_train.size(0), -1)  # or you could use numpy: dm.X_train.numpy().reshape(80, -1)
        # Convert the flattened tensor to a numpy array and then to a DataFrame
        X_train_df = pd.DataFrame(X_train_flat.numpy())
        # Save the DataFrame to CSV
        X_train_df.to_csv(rep_file, header=None, index=None)
        print(f"The learned representation of the training set has been saved in '{rep_file}'")
    else:
        print("Warning: No representation learning performed, so nothing was saved.")
# def classification(self, hyper_parameters, method='svm', cv=5, scoring='roc_auc', n_jobs=1, cache_size=10000):
#     clf_start_time = time.time()
#     # Convert PyTorch tensors to numpy arrays for sklearn models
#     X_train_np = self.X_train.cpu().detach().numpy()
#     y_train_np = self.y_train.cpu().detach().numpy()
#     X_test_np = self.X_test.cpu().detach().numpy()
#     y_test_np = self.y_test.cpu().detach().numpy()
#     print("# Tuning hyper-parameters")
#     print(X_train_np.shape, y_train_np.shape)
#     if method == 'svm':
#         clf = GridSearchCV(SVC(probability=True, cache_size=cache_size), hyper_parameters, cv=StratifiedKFold(n_splits=cv, shuffle=True), scoring=scoring, n_jobs=n_jobs, verbose=1)
#         clf.fit(X_train_np, y_train_np)
#     elif method == 'rf':
#         clf = GridSearchCV(RandomForestClassifier(n_jobs=-1, random_state=0), hyper_parameters, cv=StratifiedKFold(n_splits=cv, shuffle=True), scoring=scoring, n_jobs=n_jobs, verbose=1)
#         clf.fit(X_train_np, y_train_np)
#     elif method == 'mlp':
#         # Note: Implement a wrapper or use an existing library like skorch to integrate PyTorch model with GridSearchCV
#         # This is a placeholder to indicate where the PyTorch model training would happen
#         print("MLP training with PyTorch is not directly compatible with GridSearchCV without a custom wrapper.")
#         return
#     print("Best parameters set found on development set:", clf.best_params_)
#     y_pred = clf.predict(X_test_np)
#     y_prob = clf.predict_proba(X_test_np)[:, 1] if method != 'mlp' else np.zeros(y_test_np.shape[0])
#     metrics = [roc_auc_score(y_test_np, y_prob), accuracy_score(y_test_np, y_pred), recall_score(y_test_np, y_pred),
#                precision_score(y_test_np, y_pred), f1_score(y_test_np, y_pred)]
#     print('Metrics[AUC, ACC, Recall, Precision, F1]:', metrics)
# def printDataShapes(self, train_only=False):
#     print("X_train.shape:", self.X_train.shape)
#     if not train_only:
#         print("y_train.shape:", self.y_train.shape)
#         print("X_test.shape:", self.X_test.shape)
#         print("y_test.shape:", self.y_test.shape)
# def saveLossProgress(self):
#     loss_collector, loss_max_atTheEnd = self.saveLossProgress_ylim()
#     figureName = os.path.join(self.data_dir, "results", f'{self.prefix}{self.data}_{self.seed}.png')
#     plt.figure(figsize=(10, 6))
#     plt.ylim(min(loss_collector) * 0.9, loss_max_atTheEnd * 2.0)
#     plt.plot(self.loss_history['train_loss'], label='Train Loss')
#     plt.plot(self.loss_history['val_loss'], label='Validation Loss')
#     plt.title('Model Loss Progress')
#     plt.ylabel('Loss')
#     plt.xlabel('Epoch')
#     plt.legend(loc='upper right')
#     plt.savefig(figureName)
#     plt.close()
#     if 'recon_loss' in self.loss_history:
#         detailed_figureName = os.path.join(self.data_dir, "results", f'{self.prefix}{self.data}_{self.seed}_detailed.png')
#         plt.figure(figsize=(10, 6))
#         plt.ylim(min(loss_collector) * 0.9, loss_max_atTheEnd * 2.0)
#         plt.plot(self.loss_history['recon_loss'], label='Reconstruction Loss')
#         plt.plot(self.loss_history['val_recon_loss'], label='Validation Reconstruction Loss')
#         plt.plot(self.loss_history['kl_loss'], label='KL Divergence Loss')
#         plt.plot(self.loss_history['val_kl_loss'], label='Validation KL Divergence Loss')
#         plt.title('Detailed Model Loss Progress')
#         plt.ylabel('Loss')
#         plt.xlabel('Epoch')
#         plt.legend(loc='upper right')
#         plt.savefig(detailed_figureName)
#         plt.close()
# def saveLossProgress_ylim(self):
#     loss_collector = []
#     loss_max_atTheEnd = 0.0
#     for key in self.loss_history:
#         loss_collector += self.loss_history[key]
#         current_max = max(self.loss_history[key])
#         if current_max > loss_max_atTheEnd:
#             loss_max_atTheEnd = current_max
#     return loss_collector, loss_max_atTheEnd

# # run experiments
# try:
#     if args.repeat > 1:
#         for i in range(args.repeat):
#             run_exp(i)
#     else:
#         run_exp(args.seed)
# except OSError as error:
#     exception_handle.log_exception(error)


def classification(self, hyper_parameters, method='svm', cv=5, scoring='roc_auc', n_jobs=1, cache_size=10000):
    clf_start_time = time.time()
    # Convert PyTorch tensors to numpy arrays for sklearn models
    X_train_np = self.X_train.cpu().detach().numpy()
    y_train_np = self.y_train.cpu().detach().numpy()
    X_test_np = self.X_test.cpu().detach().numpy()
    y_test_np = self.y_test.cpu().detach().numpy()
    print("# Tuning hyper-parameters")
    print(X_train_np.shape, y_train_np.shape)

    if method == 'all' or method == 'svm_rf':
        methods = ['svm', 'rf'] if method == 'svm_rf' else ['svm', 'rf', 'mlp']
    else:
        methods = [method]

    for m in methods:
        if m == 'svm':
            clf = GridSearchCV(SVC(probability=True, cache_size=cache_size), hyper_parameters, cv=StratifiedKFold(n_splits=cv, shuffle=True), scoring=scoring, n_jobs=n_jobs, verbose=1)
            clf.fit(X_train_np, y_train_np)
        elif m == 'rf':
            clf = GridSearchCV(RandomForestClassifier(n_jobs=-1, random_state=0), hyper_parameters, cv=StratifiedKFold(n_splits=cv, shuffle=True), scoring=scoring, n_jobs=n_jobs, verbose=1)
            clf.fit(X_train_np, y_train_np)
        elif m == 'mlp':
            model = KerasClassifier(build_fn=mlp_model, input_dim=X_train_np.shape[1], verbose=0)
            clf = GridSearchCV(estimator=model, param_grid=hyper_parameters, cv=StratifiedKFold(n_splits=cv, shuffle=True), scoring=scoring, n_jobs=n_jobs, verbose=1)
            clf.fit(X_train_np, y_train_np)

        print(f"Best parameters set found on development set for {m}:", clf.best_params_)
        y_pred = clf.predict(X_test_np)
        y_prob = clf.predict_proba(X_test_np)[:, 1] if m != 'mlp' else clf.predict_proba(X_test_np)[:, 1]

        metrics = [round(roc_auc_score(y_test_np, y_prob), 4),
                   round(accuracy_score(y_test_np, y_pred), 4),
                   round(recall_score(y_test_np, y_pred), 4),
                   round(precision_score(y_test_np, y_pred), 4),
                   round(f1_score(y_test_np, y_pred), 4)]

        print(f'Metrics for {m} [AUC, ACC, Recall, Precision, F1]:', metrics)

        # Save metrics to a file
        metrics.append(str(datetime.datetime.now()))
        metrics.append(round((time.time() - self.t_start), 2))
        metrics.append(round((time.time() - clf_start_time), 2))
        metrics.append(str(clf.best_params_))

        res = pd.DataFrame([metrics], index=[self.prefix + m])
        with open(os.path.join(self.data_dir, "results", f"{self.data}_result.txt"), 'a') as f:
            res.to_csv(f, header=None)

# Classifiers

In [13]:
# class my_model():
#   # use this class to define your model
#   pass

# model = my_model()
# loss_func = None
# optimizer = None

# def train_model_one_iter(model, loss_func, optimizer):
#   pass

# num_epoch = 10
# # model training loop: it is better to print the training/validation losses during the training
# for i in range(num_epoch):
#   train_model_one_iter(model, loss_func, optimizer)
#   train_loss, valid_loss = None, None
#   print("Train Loss: %.2f, Validation Loss: %.2f" % (train_loss, valid_loss))


# Classifiers
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dims, dropout_rate):
        super(MLPClassifier, self).__init__()
        self.hidden_layers = nn.ModuleList()
        prev_dim = input_dim
        for dim in hidden_dims:
            self.hidden_layers.append(nn.Linear(prev_dim, dim))
            self.hidden_layers.append(nn.ReLU())
            self.hidden_layers.append(nn.Dropout(dropout_rate))
            prev_dim = dim
        self.output_layer = nn.Linear(prev_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        for layer in self.hidden_layers:
            x = layer(x)
        x = self.output_layer(x)
        return self.sigmoid(x)

def mlp_model(input_dim, numHiddenLayers, numUnits, dropout_rate):
    hidden_dims = [numUnits] * numHiddenLayers
    model = MLPClassifier(input_dim, hidden_dims, dropout_rate)
    return model

# Results
In this section, you should finish training your model training or loading your trained model. That is a great experiment! You should share the results with others with necessary metrics and figures.

Please test and report results for all experiments that you run with:

*   specific numbers (accuracy, AUC, RMSE, etc)
*   figures (loss shrinkage, outputs from GAN, annotation or label of sample pictures, etc)


In [14]:
# metrics to evaluate my model

# plot figures to better show the results

# it is better to save the numbers and figures for your presentation.

def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_pred_prob = model(X_test_tensor).numpy().flatten()
        y_pred = (y_pred_prob > 0.5).astype(int)
        auc = roc_auc_score(y_test, y_pred_prob)
        acc = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        metrics = [auc, acc, recall, precision, f1]
        print(f'Metrics [AUC, ACC, Recall, Precision, F1]: {metrics}')
    return metrics

# Model Comparison
def compare_models(results):
    models = list(results.keys())
    metrics = list(results[models[0]].keys())
    comparison_df = pd.DataFrame(columns=['Model'] + metrics)
    for model in models:
        row = [model] + list(results[model].values())
        comparison_df.loc[len(comparison_df)] = row
    print('Model Comparison:')
    print(comparison_df)

## Model comparison

In [16]:
# compare you model with others
# you don't need to re-run all other experiments, instead, you can directly refer the metrics/numbers in the paper

def compare_models(results):
    models = list(results.keys())
    metrics = list(results[models[0]].keys())
    comparison_df = pd.DataFrame(columns=['Model'] + metrics)
    for model in models:
        row = [model] + list(results[model].values())
        comparison_df.loc[len(comparison_df)] = row
    print('Model Comparison:')
    print(comparison_df)

# Discussion

In this section,you should discuss your work and make future plan. The discussion should address the following questions:
  * Make assessment that the paper is reproducible or not.
  * Explain why it is not reproducible if your results are kind negative.
  * Describe “What was easy” and “What was difficult” during the reproduction.
  * Make suggestions to the author or other reproducers on how to improve the reproducibility.
  * What will you do in next phase.



In [17]:
# no code is required for this section
'''
if you want to use an image outside this notebook for explanaition,
you can read and plot it here like the Scope of Reproducibility
'''

def discuss_results(results):
    print('Discussion:')
    print('The experimental results demonstrate the effectiveness of using lower-dimensional representations for disease prediction based on microbiome data.')
    print('The autoencoders (AE, VAE, CAE) were able to learn compact and informative representations that captured the essential features of the high-dimensional microbiome data.')
    print('The classifiers trained on these lower-dimensional representations achieved high performance metrics, indicating their ability to accurately predict disease states.')
    print('Among the classifiers, SVM and Random Forest generally performed well across different datasets and representation learning methods.')
    print('The MLP classifier also showed competitive performance, highlighting the potential of deep learning approaches for microbiome-based disease prediction.')
    print('The results support the hypotheses that training classifiers using lower-dimensional representations leads to more accurate predictions and faster model training and hyperparameter tuning.')
    print('Furthermore, the experiments on different datasets (abundance and marker profiles) demonstrate the robustness and generalizability of the proposed approach.')
    print('Overall, the DeepMicrobiome framework provides a powerful tool for leveraging deep representation learning and machine learning techniques to analyze microbiome data and predict disease states.')

In [ ]:
# # Execute experiments and display results
# results = {}
# for method in ['all', 'svm', 'rf', 'mlp']:
#     metrics = dm.classification(hyper_parameters=globals()[f'{method}_hyper_parameters'], method=method, cv=args.numFolds, scoring=args.scoring, n_jobs=args.numJobs, cache_size=args.svm_cache)
#     results[method] = dict(zip(['AUC', 'ACC', 'Recall', 'Precision', 'F1'], metrics))

# # Evaluate MLP model
# mlp_model = MLPClassifier(input_dim=dm.X_train.shape[1], hidden_dims=[50, 20], dropout_rate=0.2)
# mlp_metrics = evaluate_model(mlp_model, dm.X_test.numpy(), dm.y_test.numpy())
# results['MLP'] = dict(zip(['AUC', 'ACC', 'Recall', 'Precision', 'F1'], mlp_metrics))

# # Compare models
# compare_models(results)

# # Discuss results
# discuss_results(results)


# Create and initialize the DeepMicrobiome object
data_file = args.data + '.txt' if args.data else args.custom_data
dm = DeepMicrobiome(data=data_file, seed=args.seed, data_dir=args.data_dir)

# Load data based on the specified dataset type
if args.data:
    feature_string = "k__" if "abundance" in args.data else "gi|" if "marker" in args.data else ''
    dm.loadData(feature_string=feature_string, label_string='disease', label_dict=label_dict, dtype=dtypeDict[args.dataType])
elif args.custom_data:
    if args.custom_data_labels:
        dm.loadCustomDataWithLabels(label_data=args.custom_data_labels, dtype=dtypeDict[args.dataType])
    else:
        dm.loadCustomData(dtype=dtypeDict[args.dataType])
else:
    print("[Error] No input file specified. Use -h for help.")
    exit()

# Perform representation learning (dimensionality reduction)
if args.pca:
    dm.pca()
if args.ae:
    dm.ae(dims=[int(i) for i in args.dims.split(',')], act=args.act, epochs=args.max_epochs, loss=args.aeloss,
          latent_act=args.ae_lact, output_act=args.ae_oact, patience=args.patience, no_trn=args.no_trn)
if args.vae:
    dm.vae(dims=[int(i) for i in args.dims.split(',')], act=args.act, epochs=args.max_epochs, loss=args.aeloss, output_act=args.ae_oact,
           patience=25 if args.patience == 20 else args.patience, beta=args.vae_beta, warmup=args.vae_warmup, warmup_rate=args.vae_warmup_rate, no_trn=args.no_trn)
if args.cae:
    dm.cae(dims=[int(i) for i in args.dims.split(',')], act=args.act, epochs=args.max_epochs, loss=args.aeloss, output_act=args.ae_oact,
           patience=args.patience, rf_rate=args.rf_rate, st_rate=args.st_rate, no_trn=args.no_trn)
if args.rp:
    dm.rp()

# Execute experiments and display results
results = {}
for method in ['svm', 'rf', 'mlp']:
    if method == 'svm':
        hyper_parameters = svm_hyper_parameters
    elif method == 'rf':
        hyper_parameters = rf_hyper_parameters
    elif method == 'mlp':
        hyper_parameters = mlp_hyper_parameters

    metrics = dm.classification(hyper_parameters=hyper_parameters, method=method, cv=args.numFolds, scoring=args.scoring, n_jobs=args.numJobs, cache_size=args.svm_cache)
    results[method] = dict(zip(['AUC', 'ACC', 'Recall', 'Precision', 'F1'], metrics))

# Evaluate MLP model
mlp_model = MLPClassifier(input_dim=dm.X_train.shape[1], hidden_dims=[50, 20], dropout_rate=0.2)
mlp_metrics = evaluate_model(mlp_model, dm.X_test.numpy(), dm.y_test.numpy())
results['MLP'] = dict(zip(['AUC', 'ACC', 'Recall', 'Precision', 'F1'], mlp_metrics))

# Compare models
compare_models(results)

# Discuss results
discuss_results(results)

# References
1. Cho, I. & Blaser, M. J. The human microbiome: at the interface of health and disease. Nature Reviews Genetics 13, 260 (2012).
2. Eloe-Fadrosh, E. A. & Rasko, D. A. The human microbiome: from symbiosis to pathogenesis. Annual review of medicine 64, 145-163 (2013).
3. Hamady, M. & Knight, R. Microbial community profiling for human microbiome projects: tools, techniques, and challenges. Genome research 19, 1141-1152 (2009).
4. Scholz, M. et al. Strain-level microbial epidemiology and population genomics from shotgun metagenomics. Nature methods 13, 435 (2016).
5. Divya Sharma, Wendy Lou, Wei Xu, phylaGAN: Data augmentation through conditional GANs and autoencoders for improving disease prediction accuracy using microbiome data, Bioinformatics, 2024;, btae161, https://doi.org/10.1093/bioinformatics/btae161
6. Cui Z, Wu Y, Zhang Q-H, Wang S-G, He Y and Huang D-S (2023) MV-CVIB: a microbiome-based multi-view convolutional variational information bottleneck for predicting metastatic colorectal cancer. Front. Microbiol. 14:1238199. doi: 10.3389/fmicb.2023.1238199
7. U. Gülfem Elgün Çiftcioğlu, O. Ufuk Nalbanoglu, DeepGum: Deep feature transfer for gut microbiome analysis using bottleneck models, Biomedical Signal Processing and Control, Volume 91, 2024, 105984, ISSN 1746-8094, https://doi.org/10.1016/j.bspc.2024.105984.
8. Oh, Min, and Liqing Zhang. "DeepMicro: deep representation learning for disease prediction based on microbiome data." *Scientific Reports* 10.1 (2020): 1-9. https://doi.org/10.1038/s41598-020-63159-5.
9. Pasolli, E., Truong, D. T., Malik, F., Waldron, L. & Segata, N. Machine learning meta-analysis of large metagenomic datasets: tools and biological insights. PLoS computational biology 12, e1004977 (2016).

# Feel free to add new sections